In [1]:
# Libraries, Variables and Constants
import autogen, os
import json
from dotenv import load_dotenv # pip install python-dotenv

load_dotenv("./../credentials_my.env")
cache_seed = 42 # default seed is 41

In [2]:
my_models = [
    {
        'model': os.environ['GPT-4T-20240409GA-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4O-20240513-128K'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4T-VISIONPREVIEW-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {"gpt-4o-2024-05-13", "gpt-4"}
    }
)

# config_list

### [Difference between cache_seed and OpenAI's seed parameter](https://microsoft.github.io/autogen/docs/topics/llm-caching/)
OpenAI v1.1 introduced a new parameter `seed`. The difference between AutoGen's `cache_seed` and OpenAI's `seed` is AutoGen uses an explicit request cache to guarantee the exactly same output is produced for the same input and when cache is hit, no OpenAI API call will be made. OpenAI's `seed` is a best-effort deterministic sampling with no guarantee of determinism. When using OpenAI's `seed` with `cache_seed` set to `None`, even for the same input, an OpenAI API call will be made and there is no guarantee for getting exactly the same output.

In [3]:
autogen.Completion.clear_cache()

llm_config={
    "cache_seed":  cache_seed, # None, cache_seed,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
}

# llm_config

In [4]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg         = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    llm_config=llm_config
)

In [5]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name       = "assistant",
    llm_config = llm_config
)

In [6]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description in the "message" field

chat_res = user_proxy.initiate_chat(
    assistant,
    message = """What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method = "reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To compare the year-to-date (YTD) gain for META and TESLA, we need to follow these steps:

1. Get the current date.
2. Fetch the stock prices for META and TESLA at the beginning of the year and the current stock prices.
3. Calculate the YTD gain for both stocks.
4. Compare the YTD gains.

Let's start by getting the current date and fetching the stock prices. We'll use Python for this task, leveraging the `yfinance` library to get the stock data.

Here is the Python code to achieve this:

```python
# filename: ytd_gain_comparison.py

import yfinance as yf
from datetime import datetime

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')
print(f"Current Date: {current_date}")

# Define the stock symbols
stocks = ['META', 'TSLA']

# Get the stock data for the beg

## Only if we couldn't gather the stock price

In [ ]:
%%time

# followup of the previous question, if needed
user_proxy.send(
    recipient=assistant,
    message="""Please try again and use another way to gather the stock price. For example, use yfinance.""",
)

## Checks the Steps

In [7]:
i = 1
for cr in chat_res.chat_history:
    print(f'Step {i} --> {cr["content"].rstrip()[-9:]}')
    i=i+1

Step 1 --> nd TESLA.
Step 2 --> nd TESLA.
Step 3 --> yfinance'
Step 4 --> nd TESLA.
Step 5 --> : -28.79%
Step 6 --> TERMINATE


## Check the history in .cache/\<seed\>/cache.db

In [8]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

Tables in this DB: [('Settings',), ('Cache',)]

(1, '{"messages": [{"content": "You are a helpful AI assistant.\\nSolve tasks using your coding and language skills.\\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.\\nWhen using code,

## Ask an additional sub-question

In [9]:
%%time

user_proxy.send(
    recipient=assistant,
    message="""
    Plot a chart of their stock prince change YTD. Save the data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    """,
)

user_proxy (to assistant):


    Plot a chart of their stock prince change YTD. Save the data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot a chart of the stock price changes YTD for META and TESLA, and save the data to a CSV file and the plot to a PNG file, we can extend the previous Python script. We'll use the `matplotlib` library for plotting and `pandas` for handling the CSV file.

Here is the complete script:

```sh
# filename: plot_stock_price_ytd.sh

# Install necessary libraries
pip install yfinance matplotlib pandas

# Save the Python script to a file
cat << 'EOF' > plot_stock_price_ytd.py
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')
print(f"Current Date: {current_date}")

# Define the stock symbols

# Hic Sunt Leones